In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_cloud = os.path.join(folder_AGVA, 'classified images', 'S2_Cloud_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values


print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

3031


In [ ]:
skip = 0
for i in range(len(rgis_to_analyze)):
    
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]

    # options for skipping
#     if (i+1)<296: continue #everything done to here
#     if (i+1)>400: continue #everything done to here 
#     if (i+1) not in [405,406,407,408,420, 291]: continue
#     if (i+1) not in [280, 290]: continue
#     if rgiid != "RGI60-01.03215": continue
#     if skip: continue

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]
    if ga<925: continue
    
    # print progress
#     if (i+1)!=4: continue
#     if (i+1)!=11: continue
#     if (i+1)>1: continue
    

    if validation_only:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    else:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
        
    # check if this glacier has been run already, skip if so
    temp_path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_2022_daily_AAs.nc")
#     if os.path.exists(temp_path):
#         continue
        
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
        
#     break
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry

    # single_geometry = single_geometry.buffer(-100) #what if we buffer out the exterior 100 meters of the glacier
    
    # open glacier mask
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier 
    
    # open the classification data. only chunk (use dask) if the glacier is "big"
    file_name = f"S2_{rgiid}_2018-01-01_2023-01-01"
    if ga>500:
        xr_class = riox.open_rasterio(os.path.join(folder_class, f"{file_name}.tif"), chunks={'band':1})#.chunk({'band':-1, 'y':'auto', 'x':'auto'})
    elif ga>100:
        xr_class = riox.open_rasterio(os.path.join(folder_class, f"{file_name}.tif"), chunks={'band':10})#.chunk({'band':-1, 'y':'auto', 'x':'auto'})
    else:
        xr_class = riox.open_rasterio(os.path.join(folder_class, f"{file_name}.tif"))
        
    # load metadata csv, convert date to datetimes
    meta_fp = os.path.join(folder_meta, f"{file_name}.csv")
    meta_df = pd.read_csv(meta_fp)
    
    # format time axis to pandas datetime, like xarray wants
    datetimes = pd.to_datetime([f"{str(i)[:4]}-{str(i)[4:6]}-{str(i)[6:]}" for i in meta_df['date']])
    datetimes_unique = datetimes.unique()
    xr_class = xr_class.rename({"band":"time"})
    xr_class['time'] = datetimes

    # merge images on same day, if there are repeated dates
    if len(datetimes)!=len(datetimes.unique()):
        xr_class = xr_class.where(xr_class<20, 0).groupby('time').max('time')
    else:
        xr_class = xr_class.where(xr_class<20, 0)

    # get these merged dates
    datetimes_merged = xr_class.time.values

    ### create binary mask of useable and unuseable data, use it to mask the xr_class to only this area
    bad_classes = [0] #class 5 is shadow. we will call these area unusable
    good_classes = [1,2,3,4,5,6] #snow,firn,ice,debris,water are usable areas
    # usable = xr.where( xr_class.isin(good_classes), 1, 0)

    # count total number of pixels on the glacier surface, based on the glacier rgi area
#     glacier_pixels = int(ga * (1000*1000) / (10*10))
    glacier_pixels = glacier_mask.sum().values
#     print('here2')
    
    # now we can mask out unusable areas in each time step
    xr_class = xr.where(xr_class.isin(good_classes), xr_class, 0) #.sel(time=good_times) # note we subset to only the good days

    # make df with zeros that can be filled in later
    obs_df = pd.DataFrame({'Date':datetimes_merged, 'observed_initial':[0 for i in datetimes_merged]})

    # at this point, xr_class is 0 off-glacier, 0 in shadow/cloud, and 1-6 in usable areas
    # save one year at a time
    for y in [2018,2019,2020,2021,2022]:
        if ga>200: print(y)
#         if y==2018: continue
#         if y<2020: continue
        path_save = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs.nc")
        
        # new method we only use months 5-11
        save_xr = xr_class.sel(time=slice(f"{y}-5-01", f"{y}-11-30")).astype('uint8').rename('class')

        # specify compression/encoding
        encoding = {"class":{"zlib": True}}#, "spatial_ref":{"zlib": False}}

        # save
        save_xr.to_netcdf(path_save, encoding=encoding)
        
#         # get dates
#         dates_this_year = save_xr.time.values
        
#         ### then calculate % useable on each date, add to csv to save
#         # if this is a big glacier, then lets reopen to calculate this
#         # if REALLY big, then coarsen before computing
#         if ga>3000:
#             continue
            
#         elif ga>500:                   
#             # open data
#             xr_to_use = xr.open_dataset(path_save, chunks={'band':1, 'y':-1, 'x':-1})['class']

#             # coarsen and save
#             path_temp = os.path.join(folder_save, 'temp.tif')
#             usable = xr.where(xr_to_use>0, 1, 0).astype("uint8").coarsen({"x":5, "y":5}, boundary="trim").median(skipna=True).astype("uint8").compute()
#             usable.rio.to_raster(path_temp, dtype='uint8')
            
#             # reopen and calculate percent useable on each day
#             usable = riox.open_rasterio(path_temp)
#             percent_usable_by_time = (usable.sum(dim=['x','y'])).compute().values
            
#             # scale correctly
#             percent_usable_by_time = percent_usable_by_time*25/glacier_pixels
        
#         elif ga>150:
#             xr_to_use = xr.open_dataset(path_save, chunks={'band':1, 'y':-1, 'x':-1}).to_array()
#             percent_usable_by_time = (xr.where(xr_to_use>0, 1, 0).sum(dim=['x','y'])/glacier_pixels).compute().values[0]
        
#         else:
#             xr_to_use = save_xr
#             percent_usable_by_time = (xr.where(xr_to_use>0, 1, 0).sum(dim=['x','y'])/glacier_pixels).compute().values
        
        
        
# #         print(obs_df['Date'].isin(dates_this_year).sum())
# #         print(percent_usable_by_time)
        
#         obs_df.loc[obs_df['Date'].isin(dates_this_year), 'observed_initial'] = percent_usable_by_time.round(4)
    
#     if ga<3000:
#         out_path_csv = os.path.join(folder_save, 'Daily AAs', 'observed', f"S2_{rgiid}_observed.csv")
#         obs_df.to_csv(out_path_csv, index=False)
    print("done")


Starting 3023 of 3031: RGI60-01.17614  925.341 km2
2018
2019
2020
2021
2022


In [ ]:
xr_class=0
xr_to_use = 0